# NSD Final Project

## Title: 

### Team member: 李心怡，尉银杰

$$
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
%\newcommand{\Pr}{\mathrm{Pr} }
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\RR}{\mathbb{R} }
\newcommand{\EE}{\mathbb{E} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\sgn}{\mathrm{sgn} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
%\newcommand{\HS}{\mathrm{HS} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
%\newcommand{\LL}{\mathrm{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\xx}{\mathcal{X} }
\newcommand{\nn}{\mathcal{NN} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\p}{\partial}
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bi}{\begin{itemize}}
\newcommand{\ei}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\cO}{\mathcal{O}}
\newcommand{\cF}{\mathcal{F}}
\newcommand{\cL}{\mathcal{L}}
\newcommand{\BS}{\text{BS}}
$$

### Abstract

### Table of content (optional)

### Introduction

- Hedging has always been a main concern for the sellside. Classic approach of hedging requires a specific model for the dynamics of the hedging instruments, e.g. Black-Scholes model and its variants, which the real data may not follow. Thus, accuracy of such hedging strategy is impaired once the underlying model is proven wrong.

- With the rise of modern machine learning techniques, more and more researchers tried to implement a model-free method which allows for theoretically perfect hedging. In addition, it is easier to add market frictions to such strategy. In our project, we will introduce a reinforcement learning method from the paper "Deep Hedging" (Buehler et al, 2019), and try to reproduce its experiment result.

### Setting 1 - Discrete-time Market with Friction

#### Discrete-time
Below are some basic notations for a discrete-time market.

- $T$: finite time horizon, maximum maturity of all instruments.

- $0=t_0<t_1<\cdots <t_n=T$: trading dates.

- $I_k \in \RR^r$: any NEW market information at time $t_k$.

- $\ff_k$: any market information at time $t_k$.

- $S = (S_k)_{k=0,1,\dots ,n}$, $S_k \in \RR_d$: prices of $d$ hedging instruments adapted to filtration $\ff$, including primary assets like equities and secondary assets like options.

- $Z$: agent's liabilities, which is a contingent claim and is adapted to filtration $\ff$.

- $\delta = (\delta_k)_{k=0,1,\dots ,n}$, $\delta_k \in \RR_d$: hedging strategy, where $\delta_k^i$ denotes agent's holdings of the $i$th asset at time $t_k$, and we stipulate that $\delta_{-1}=\delta_n=0$.

- $p_0$: addtional cash injected to the agent's portfolio at time $0$.



#### Frictions

We define the following variables to allow frictions in the market:

- $\hh^u$: unconstrained set of hedging strategy $\delta$.

- $\hh_k^c$: constrained set of the holdings at time $t_k$ due to liquidity, asset availability and trading restrictions.

- $H_k: \RR^{d(k+1)}\rightarrow\RR^{d}$: function that computes constrained holdings $\delta_k^c\in \hh_k^c$ from unconstrained holdings $\delta_k^u\in \hh^u$ and constrained holdings $\{\delta_i^c\in \hh_i^c\}$ for $i \in \{0,1,...,k-1\}$.

- $H$: constrained "projection" from $\hh^u$ to $\hh_k^c$. It can be defined successively as $(H\circ\delta_k^u)_k = H_k\{(H\circ\delta_0^u)_0,(H\circ\delta_1^u)_1,...,(H\circ\delta_{k-1}^u)_{k-1}, \delta_k^u\}$

- $c_k(\delta_k-\delta_{k-1})$: cost of trading incurred at time $t_k$ for changing the holdings from $\delta_k$ to $\delta_{k-1}$. Typically it takes one of the following forms:
<ul>
    
- Proportional transaction cost: $c_k(n) = \sum_{i=1}^d c_k^iS_k^i|n^i|$

- Fixed transaction cost: $c_k(n) = \sum_{i=1}^d c_k^i1_{|n^i|\ge \epsilon}$

- Complex cross asset cost: such as the cost of volatility
</ul>

- $C_T(\delta) = \sum_{k=1}^T c_k(\delta_k-\delta_{k-1})$: total transaction costs of trading a strategy $\delta$ up to maturity $T$.




#### Portfolio Value

Given the notations above, we have the terminal value of the agent's portfolio:
$$PL_T(Z,p_0,\delta) = -Z+p_0+(\delta\cdot S)_T-C_T(\delta)$$

### Setting 2 - Hedging under Convex Risk Measures

In a complete market without frictions, there exists a unique hedging strategy $\delta$ and a fair price $p_0$ such that $PL_T=0$ holds a.s. However, that is not the case in an incomplete market with friction. Thus, we have to first define a optimality criterion (here we choose convex risk measures). Then we define the optimal hedging strategy as a minimizer of that criterion.

#### Definition of convex risk measure
$\rho: \xx\rightarrow\RR$ is a convex risk measures if it satisfies:

- Monotone decreasing: if $X_1\ge X_2$ then $\rho(X_1)\le \rho(X_2)$.

- Convex: $\rho(aX_1+(1-a)X_2)\le a\rho(X_1)_(1-a)\rho(X_2)$ for $a\in [0,1]$

- Cash-Invariant: $\rho(X+c) = \rho(X)-c$ for $c\in\RR$
    
where $X_1, X_2, X\in\xx$ are asset positions, and comparisons between asset positions represents agent's preference. If $\rho(0)=0$ we call it normalized.

#### Optimal hedging strategy
Given the convex risk measure, our optimality problem for hedging becomes
$$
\pi(-Z) =\underset{\delta\in\hh^c}\inf\rho(-Z+(\delta\cdot S)_T-C_T(\delta)),
$$
where $\hh^c$ is the constrained set for trading strategy $\delta$.

So the optimal hedging policy $\delta$ is defined as the minimizer of $\pi(-Z)$.

#### Remark
Some literature uses risk adjusted return $\rho(X)=\EE[X]-\lambda Var(X)$ as the objective function, which is an ituitive measure of hedging performance. However it does not satisfy the monotone decreasing property, so it is not a convex risk measure. Therefore we will not use it in this project.

### Setting 3 - Pricing under Convex Risk Measures

In an incomplete market with friction, a fair price of $Z$ no longer exists, so we need to define an indifference price $p(Z)$.

#### Definition of indifference price
$p(Z)$ is the indifferent price of $Z$ that satisfies:
$$
\pi(-Z+p(Z))=\pi(0)\\
$$
By cash-invariant property of $\pi$ (which can be deduced from the cash-invariant property of $\rho$):
$$
p(Z) = \pi(-Z)-\pi(0)
$$

#### Remark
Financial meaning of indifference price: the minimal amount of cash we need to charge in order to make us indifferent between taking the position $-Z$ and not doing so under risk measure $\rho$.


### Setting 4 - Conditional Value at Risk (CVar)

We will choose the convex risk measure $\rho$ to be conditional value at risk (CVar, a.k.a. average value at risk or expected shortfall) in our numerical experiments. 

#### Definition of CVar
For random variable $X$, CVar is defined as follows:
$$
CVar(X)= \frac1{1-\alpha}\int_0^{1-\alpha}VaR_\gamma(X)dX
$$
where $\alpha \in [0,1)$ is known as level of risk aversion, and $VaR_\gamma(X) = \inf\{m\in\RR:\PP(X<-m)\le \gamma\}$.

#### Remark
Finacial meaning of CVar:
- $X$ is the return.
- Value at risk ($VaR_\gamma$) means that, with probability $\gamma$, we will get a return worse than $-VaR_\gamma$. In contrast to other risk measure like variance, it mainly focuses on the risk of potential shortfalls, and ignores how good returns can be for the top $100(1-\gamma)$ percent cases. 
- Conditional value at risk (CVar) takes the average of $VaR$ from $\gamma=0$ to $\gamma=1-\alpha$. It focuses the distribution of the bottom $100(1-\alpha)$ percent returns. 
<ul>
- For $\alpha$ close to 1 (i.e. $1-\alpha$ close to 0), we focus more on extremely bad cases and favor those strategies that give us less extremely bad returns (but also potentially less good returns). 
- For $\alpha$ close to 0 (i.e. $1-\alpha$ close to 1), CVar converges to expected return, i.e. we only concern about the average return regardless of its distribution (risk).
</ul>
Therefore $\alpha$ can ituitively be thought of as level of risk aversion (larger the $\alpha$, more risk averse).

### Neural Networks 1 - Definition of neural networks

A function $F:\RR^{N_0}\rightarrow\RR^{N_L}$ defined as
$$
F(x)=W_L\circ F_{L-1}\circ...\circ F_0
$$
is a feed forward neural network, where

- $L$: number of layer in neural network.
- $N_l$, $l\in \{0,1,...,L\}$: dimention of the $l$th layer. ($N_0$ input dimension, $N_L$ output dimension)
- $W_l(x) = A_lx+b_l$: affine function transforming the input $x$ from dimension $N_{l-1}$ to $N_l$.
- $\sigma:\RR\rightarrow\RR$: non-linear function (activation function) applied componentwise.
- $F_l(x) = \sigma\circ W_l(x)$: the $l$th layer in neural network.

#### Remark
- We denote by $\nn_{M,d_0,d_1}$ the set of neural networks that has $M$ non-zero parameters, input dimension $d_0$, output dimension $d_1$.
- Literature proved that neural networks with infinity number of parameters can approximate multivariate functions arbitrarily well, which serves as theoretical fundation for using neural networks to approximate our hedging strategy $\delta$.

### Neural Networks 2 - Semi-recurrent Neural Networks
We will use the following semi-recurrent neural network to approximate the optimal hedging strategy $\delta$:
$$
\delta_k = F_k(I_0,...,I_k,\delta_{k-1}) 
$$
for $k\in\{0,1,...,n-1\}$, where
$$
F_k\in \nn_{M,r(k+1)+d,d}.
$$

As a reminder:
- n is the number of trading dates, i.e. the time when we change our holdings.
- r is the dimension of new information $I_k$.


#### Remark
- Financial meaning of semi-recurrent neural network: at each time $t_k$, we input not only all the information available by the time $t_k$, but also our holdings $\delta_{k-1}$ which is computed previously from the same neural network (except $k=0$ when $\delta_{-1}$ is set to 0).

- In numerical experiment below, we will simplify the semi-recurrent neural network to
$$
\delta_k = F_k(I_k,\delta_{k-1}) ,
$$
i.e. our network only takes new information and current holdings as input and ignores the historical infomation.

### Neural Networks 3 - Loss Function for Neural Networks

In previous cells we have written the following objective function:
$$
\pi(-Z) =\underset{\delta\in\hh^c}\inf\rho(-Z+(\delta\cdot S)_T-C_T(\delta)).
$$

To make the constrained function an unconstrained one, we use the "projection" defined previously:
$$
\pi(-Z) =\underset{\delta\in\hh^u}\inf\rho(-Z+(H\circ\delta\cdot S)_T-C_T(H\circ\delta)).
$$

Now that $\delta$ is the output of the semi-recurrent neural network with $M$ parameters, we define $\hh_M$ the set of possible $\delta$ as
\begin{eqnarray*}
\hh_M
&=&\{
(\delta_k)_{k=0,1,...,n-1}\in\hh^u: 
\delta_k = F_k(I_0,...,I_k,\delta_{k-1})&,&
F_k\in \nn_{M,r(k+1)+d,d}
\}\\
&=&\{
(\delta_k^\theta)_{k=0,1,...,n-1}\in\hh^u: 
\delta_k^\theta = F^{\theta_k}(I_0,...,I_k,\delta_{k-1}^\theta)&,&
\theta_k\in \Theta_{M,r(k+1)+d,d}
\}
\end{eqnarray*}
where
- $\theta_k$ denotes network parameters for $F_k$. Note that $\theta_k$ is different for each $k$.
- $\Theta_{M,r(k+1)+d,d}$ denotes the parameter space for $\nn_{M,r(k+1)+d,d}$.
- $\delta_k^\theta$ denotes the holdings at time $t_k$ computed given $\theta_k\in \Theta_{M,r(k+1)+d,d}$.

Therefore, we have the final form of objective function
\begin{eqnarray*}
\pi^M(-Z)
&=&
\underset{\delta\in\hh_M}\inf\rho(-Z+(H\circ\delta\cdot S)_T-C_T(H\circ\delta)).\\
&=&
\underset{\theta\in\Theta_M}\inf\rho(-Z+(H\circ\delta^\theta\cdot S)_T-C_T(H\circ\delta^\theta))\\
\end{eqnarray*}
where $\Theta_M=\prod_{k=0}^{n-1}\Theta_{M,r(k+1)+d,d}$,

and solving this function is the same as finding finite dimensional parameter $\theta$ for our NN that minimize the following loss function
$$
L(\theta)=\rho(-Z+(H\circ\delta^\theta\cdot S)_T-C_T(H\circ\delta^\theta))
$$

#### Remark
- Because the neural network can approximate arbitrarily well when $M\rightarrow\infty$, $\hh_M\approx\hh^u$. So the optimal strategy computed using NN should be close to the real optimal solution.

- In practice we use gradient descent of the loss function w.r.t. the network parameters to training network. However, our network output $\delta^\theta$ is always transformed by the "projection" function $H$ in the loss function. As a result, if $\delta^\theta\notin \hh^c$, loss function can not pass this information through gradient descent to network parameters. Therefore, when $H\circ\delta^\theta=\delta^\theta$ is satisfied for all $\delta^\theta\in\hh^c$, we typically add a panelty term to the loss function:
$$
L(\theta)=\rho(-Z+(H\circ\delta^\theta\cdot S)_T-C_T(H\circ\delta^\theta))-\gamma\lVert \delta^\theta-H\circ\delta^\theta\rVert_1
$$
with $\gamma\gg 0$.

### Numerical Experiments 1: Experiment Design

We will reproduce in total 2 numerical experiments:

- 

### Numerical Experiments 2:  

### Conclusion and discussions

### Reference (optional)